In [166]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

## Loading the 12 split CSVs and combining back into a single dataframe

- As the original CSV has been split into 12, only the first file contains the proper column names.
- Therefore, we need to ensure that files 2 through to 12 are read in a way where their first row isn't treated as the column names.

In [167]:
# Load CSV, need to go up one folder level then back down
df = pd.read_csv('Raw Data/all_energy_statistics1.csv')
column_names = df.columns.to_list()

In [168]:
# Iterating through every subsquent CSV and appending to first CSV
file_number = 2
for i in np.arange(11):
    # print(f'About to load file number {file_number}') # for debugging/ sanity check
    df_ = pd.read_csv(f'Raw Data/all_energy_statistics{file_number}.csv',header=None) # header=None ensures the column names are set to numbers i.e. not the first row
    df_.columns = column_names
    df = df.append(df_)
    file_number+=1 # used for selecting correct file name in each iteration

In [169]:
df

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.0,NaN,additives_and_oxygenates
...,...,...,...,...,...,...,...
89478,Viet Nam,Electricity - total wind production,2012,"Kilowatt-hours, million",92.0,1.0,wind_electricity
89479,Viet Nam,Electricity - total wind production,2011,"Kilowatt-hours, million",87.0,NaN,wind_electricity
89480,Viet Nam,Electricity - total wind production,2010,"Kilowatt-hours, million",50.0,NaN,wind_electricity
89481,Viet Nam,Electricity - total wind production,2009,"Kilowatt-hours, million",10.0,NaN,wind_electricity


Now the dataframe has been reassembled, but did the splitting of the file and then recombining it retain the correct number of rows?

In [174]:
row_num_kaggle = 1189482 # https://www.kaggle.com/alexanderklarge/checking-out-data-set-for-seeyoudata-project
row_num_here = df.shape[0]

# Assert statement. Nice and clean, although no option to make it print something out to confirm that you've passed (the lack of an AssertionError is your confirmation)
assert row_num_kaggle == row_num_here, 'Dataframes don\'t match!'

# Ternary statement. Essentially a 1 line "if:else" statement, lets you do something if or else.
'Dataframe is the right shape!' if row_num_kaggle == row_num_here else 'Dataframe is the wrong shape!'

'Dataframe is the right shape!'

## Cleaning the dataframe

- First of all, I'd like to take the commodity_transaction column and turn it into a separate column per commodity type

In [175]:
df.head(3)

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates


As you can see, the dataframe is very concise with only 7 columns. However the commodity_transaction column contains __many__ different values. It would be much better if each of these were their own column.

In [176]:
# Creating a groupby
# Problem here is losing string columns i.e. unit, but I suppose I can rejoin them on later
cols_of_interest = ['country_or_area','year','commodity_transaction','quantity']
df_groupby = df[cols_of_interest].groupby(['country_or_area','year','commodity_transaction']).sum()
df_groupby.head(3)

quantity
country_or_area year commodity_transaction                                       
Afghanistan     1990 Aviation gasoline - Consumption by transport             3.0
                     Aviation gasoline - Consumption in domestic avi...       3.0
                     Aviation gasoline - Final consumption                    3.0

In [177]:
# Using the mystical unstack method to magically turn the commodity_transaction column into a column per commodity_transaction type!
df_groupby = df_groupby.unstack()

In [178]:
df_groupby.head(3)

quantity  \
commodity_transaction Additives and Oxygenates - Exports   
country_or_area year                                       
Afghanistan     1990                                 NaN   
                1991                                 NaN   
                1992                                 NaN   

                                                          \
commodity_transaction Additives and Oxygenates - Imports   
country_or_area year                                       
Afghanistan     1990                                 NaN   
                1991                                 NaN   
                1992                                 NaN   

                                                             \
commodity_transaction Additives and Oxygenates - Production   
country_or_area year                                          
Afghanistan     1990                                    NaN   
                1991                                    NaN   
                1992                                    NaN   

                                                                              \
commodity_transaction Additives and Oxygenates - Receipts from other sources   
country_or_area year                                                           
Afghanistan     1990                                                 NaN       
                1991                                                 NaN       
                1992                                                 NaN       

                                                                \
commodity_transaction Additives and Oxygenates - Stock changes   
country_or_area year                                             
Afghanistan     1990                                       NaN   
                1991                                       NaN   
                1992                                       NaN   

                                                                      \
commodity_transaction Additives and Oxygenates - Total energy supply   
country_or_area year                                                   
Afghanistan     1990                                             NaN   
                1991                                             NaN   
                1992                                             NaN   

                                                                 \
commodity_transaction Additives and Oxygenates - Transformation   
country_or_area year                                              
Afghanistan     1990                                        NaN   
                1991                                        NaN   
                1992                                        NaN   

                                                                                   \
commodity_transaction Additives and Oxygenates - Transformation in oil refineries   
country_or_area year                                                                
Afghanistan     1990                                                 NaN            
                1991                                                 NaN            
                1992                                                 NaN            

                                                                                  \
commodity_transaction Additives and Oxygenates - transfers and recycled products   
country_or_area year                                                               
Afghanistan     1990                                                 NaN           
                1991                                                 NaN           
                1992                                                 NaN           

                                                                                  \
commodity_transaction Animal waste - Consumption by commerce and public services   
country_or_area year                             

In [180]:
# Unstack leaves you with an unpleasent nested/ multiindex column structure, which might have some utility I'm not aware of, but I'd rather remove for now at least
df_groupby.columns

MultiIndex([('quantity', ...),
            ('quantity', ...),
            ('quantity', ...),
            ('quantity', ...),
            ('quantity', ...),
            ('quantity', ...),
            ('quantity', ...),
            ('quantity', ...),
            ('quantity', ...),
            ('quantity', ...),
            ...
            ('quantity', ...),
            ('quantity', ...),
            ('quantity', ...),
            ('quantity', ...),
            ('quantity', ...),
            ('quantity', ...),
            ('quantity', ...),
            ('quantity', ...),
            ('quantity', ...),
            ('quantity', ...)],
           names=[None, 'commodity_transaction'], length=2452)

In [181]:
df_groupby.columns = df_groupby.columns.droplevel()

In [182]:
df_groupby.columns

Index(['Additives and Oxygenates - Exports',
       'Additives and Oxygenates - Imports',
       'Additives and Oxygenates - Production',
       'Additives and Oxygenates - Receipts from other sources',
       'Additives and Oxygenates - Stock changes',
       'Additives and Oxygenates - Total energy supply',
       'Additives and Oxygenates - Transformation',
       'Additives and Oxygenates - Transformation in oil refineries',
       'Additives and Oxygenates - transfers and recycled products',
       'Animal waste - Consumption by commerce and public services',
       ...
       'White spirit and special boiling point industrial spirits - Transfers and recycled products',
       'White spirit and special boiling point industrial spirits - Transformation',
       'White spirit and special boiling point industrial spirits - Transformation in petrochemical plants',
       'White spirit and special boiling point industrial spirits - consumption by other industries and construction',
   

In [183]:
# We now have a dataframe which is easier to navigate, although ideally I'd like it not to be a groupby at all
df_groupby

commodity_transaction  Additives and Oxygenates - Exports  \
country_or_area year                                        
Afghanistan     1990                                  NaN   
                1991                                  NaN   
                1992                                  NaN   
                1993                                  NaN   
                1994                                  NaN   
...                                                   ...   
Zimbabwe        2010                                  NaN   
                2011                                  NaN   
                2012                                  NaN   
                2013                                  NaN   
                2014                                  NaN   

commodity_transaction  Additives and Oxygenates - Imports  \
country_or_area year                                        
Afghanistan     1990                                  NaN   
                1991                                  NaN   
                1992                                  NaN   
                1993                                  NaN   
                1994                                  NaN   
...                                                   ...   
Zimbabwe        2010                                  NaN   
                2011                                  NaN   
                2012                                  NaN   
                2013                                  NaN   
                2014                                  NaN   

commodity_transaction  Additives and Oxygenates - Production  \
country_or_area year                                           
Afghanistan     1990                                     NaN   
                1991                                     NaN   
                1992                                     NaN   
                1993                                     NaN   
                1994                                     NaN   
...                                                      ...   
Zimbabwe        2010                                     NaN   
                2011                                     NaN   
                2012                                     NaN   
                2013                                     NaN   
                2014                                     NaN   

commodity_transaction  Additives and Oxygenates - Receipts from other sources  \
country_or_area year                                                            
Afghanistan     1990                                                 NaN        
                1991                                                 NaN        
                1992                                                 NaN        
                1993                                                 NaN        
                1994                                                 NaN        
...                                                                  ...        
Zimbabwe        2010                                                 NaN        
                2011                                                 NaN        
                2012                                                 NaN        
                2013                                                 NaN        
                2014                                                 NaN        

commodity_transaction  Additives and Oxygenates - Stock changes  \
country_or_area year                                              
Afghanistan     1990                                        NaN   
                1991                                        NaN   
                1992                                        NaN   
                1993                                        NaN   
                1994                                        NaN   
...                                                     

In [184]:
# Would ideally like the index to not be a multiindex
df_groupby.index

MultiIndex([('Afghanistan', 1990),
            ('Afghanistan', 1991),
            ('Afghanistan', 1992),
            ('Afghanistan', 1993),
            ('Afghanistan', 1994),
            ('Afghanistan', 1995),
            ('Afghanistan', 1996),
            ('Afghanistan', 1997),
            ('Afghanistan', 1998),
            ('Afghanistan', 1999),
            ...
            (   'Zimbabwe', 2005),
            (   'Zimbabwe', 2006),
            (   'Zimbabwe', 2007),
            (   'Zimbabwe', 2008),
            (   'Zimbabwe', 2009),
            (   'Zimbabwe', 2010),
            (   'Zimbabwe', 2011),
            (   'Zimbabwe', 2012),
            (   'Zimbabwe', 2013),
            (   'Zimbabwe', 2014)],
           names=['country_or_area', 'year'], length=5568)

In [185]:
# Apparently this will turn the multiindex into normal columns:
df_groupby.reset_index(inplace=True)

In [186]:
df_groupby

commodity_transaction,country_or_area,year,Additives and Oxygenates - Exports,Additives and Oxygenates - Imports,Additives and Oxygenates - Production,Additives and Oxygenates - Receipts from other sources,Additives and Oxygenates - Stock changes,Additives and Oxygenates - Total energy supply,Additives and Oxygenates - Transformation,Additives and Oxygenates - Transformation in oil refineries,...,White spirit and special boiling point industrial spirits - Transfers and recycled products,White spirit and special boiling point industrial spirits - Transformation,White spirit and special boiling point industrial spirits - Transformation in petrochemical plants,White spirit and special boiling point industrial spirits - consumption by other industries and construction,White spirit and special boiling point industrial spirits - final consumption,White spirit and special boiling point industrial spirits - production from plants,White spirit and special boiling point industrial spirits - production from refineries,Wind – Autoproducer,Wind – Main activity,animal waste - Transformation
0,Afghanistan,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,21.0,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5563,Zimbabwe,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5564,Zimbabwe,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5565,Zimbabwe,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5566,Zimbabwe,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [187]:
# However we now have a useless index column, so I'll set it to country_or_area instead
df_groupby.set_index('country_or_area',inplace=True)

In [188]:
df_groupby

commodity_transaction,year,Additives and Oxygenates - Exports,Additives and Oxygenates - Imports,Additives and Oxygenates - Production,Additives and Oxygenates - Receipts from other sources,Additives and Oxygenates - Stock changes,Additives and Oxygenates - Total energy supply,Additives and Oxygenates - Transformation,Additives and Oxygenates - Transformation in oil refineries,Additives and Oxygenates - transfers and recycled products,...,White spirit and special boiling point industrial spirits - Transfers and recycled products,White spirit and special boiling point industrial spirits - Transformation,White spirit and special boiling point industrial spirits - Transformation in petrochemical plants,White spirit and special boiling point industrial spirits - consumption by other industries and construction,White spirit and special boiling point industrial spirits - final consumption,White spirit and special boiling point industrial spirits - production from plants,White spirit and special boiling point industrial spirits - production from refineries,Wind – Autoproducer,Wind – Main activity,animal waste - Transformation
country_or_area,,,,,,,,,,,,,,,,,,,,,
Afghanistan,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,21.0,NaN,NaN,NaN,NaN,NaN
Afghanistan,1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN,NaN,NaN
Afghanistan,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN
Afghanistan,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN
Afghanistan,1994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zimbabwe,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zimbabwe,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zimbabwe,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The dataframe is now in a nice & clean (flat?) format for investigating as normal. 

(This also seems to me like it's ready to be put into a SQL table, which will be a cool project. If we can get this data hosted in a SQL server, we can then connect it to a website. Using PHP and JavaScript, you could have user dropdowns to select for example country x in year y, and then return those values in a table, or maybe even make a live graph)

## Making the data easier to investigate - combining columns into categories

- To make the data easier to handle, I'd like to take the commodity types and combine by category of commodity
- The categories may be easy to assign based on the unit provided in the dataset

### Step 1: making a dictionary of unit:[commondity_columns]

In [193]:
# I want to have each unit, and then with it a list of all the columns that have that unit
# The best way I can think to do that is with a nested dictionary
unit_and_commod_dict = {'test':'test2'} # instantiating the dictionary

In [194]:
for i in df.unit.unique():
    print(i)
    
    # Creating a list of columns for a specific unit
    list_of_commodities_for_specific_unit = df[df['unit']==i]['commodity_transaction'].unique() # this is apparently a set
    list_of_commodities_for_specific_unit = list(list_of_commodities_for_specific_unit) # this is now a list, so I won't get an error
    list_of_commodities_for_specific_unit.insert(0,'year') # adding year at the start to help me later on, as I'm going to want to select all the commodity columns but also the year column
    
    # Inserting into the dicitonary. Key = unit, value = list of columns (and the year column at the start)
    unit_and_commod_dict[i] = list_of_commodities_for_specific_unit
    #print('\n')

Metric tons,  thousand
Terajoules
Kilowatts,  thousand
Kilowatt-hours, million
Cubic metres, thousand
Metric Tons


In [195]:
del unit_and_commod_dict['test']

In [196]:
for k,v in unit_and_commod_dict.items():
    print(k)

Metric tons,  thousand
Terajoules
Kilowatts,  thousand
Kilowatt-hours, million
Cubic metres, thousand
Metric Tons


### Step 2: making a dataframe of just the units

In [197]:
# I've heard NAs referenced to as "contagious" i.e. they can accidentally (?) turn other things into NA, and I was having issues before adding this bit in
df_groupby.fillna(0,inplace=True)

In [198]:
df_groupby

commodity_transaction,year,Additives and Oxygenates - Exports,Additives and Oxygenates - Imports,Additives and Oxygenates - Production,Additives and Oxygenates - Receipts from other sources,Additives and Oxygenates - Stock changes,Additives and Oxygenates - Total energy supply,Additives and Oxygenates - Transformation,Additives and Oxygenates - Transformation in oil refineries,Additives and Oxygenates - transfers and recycled products,...,White spirit and special boiling point industrial spirits - Transfers and recycled products,White spirit and special boiling point industrial spirits - Transformation,White spirit and special boiling point industrial spirits - Transformation in petrochemical plants,White spirit and special boiling point industrial spirits - consumption by other industries and construction,White spirit and special boiling point industrial spirits - final consumption,White spirit and special boiling point industrial spirits - production from plants,White spirit and special boiling point industrial spirits - production from refineries,Wind – Autoproducer,Wind – Main activity,animal waste - Transformation
country_or_area,,,,,,,,,,,,,,,,,,,,,
Afghanistan,1990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0
Afghanistan,1991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0
Afghanistan,1992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0
Afghanistan,1993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0
Afghanistan,1994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zimbabwe,2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zimbabwe,2011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zimbabwe,2012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [199]:
############# DEBUGGING AS FOR SMOE REASON ALL THE VALUES GET DELETED
df_units = df_groupby.copy()

In [200]:
units = my_dict.keys()
units = list(units)

In [201]:
cols_of_interest = units.copy()
cols_of_interest.insert(0,'year')

In [202]:
for k, v in unit_and_commod_dict.items():
    df_units[k] = df_units[v[1:]].sum(axis=1) # aiming to avoid including the year column in the total

In [205]:
df_units = df_units[cols_of_interest]

In [206]:
df_units

commodity_transaction,year,"Metric tons, thousand",Terajoules,"Kilowatts, thousand","Kilowatt-hours, million","Cubic metres, thousand",Metric Tons
country_or_area,,,,,,,
Afghanistan,1990,1.094542e+06,3961093.0,1976.0,8065.0,3679.368000,0.0
Afghanistan,1991,8.042161e+05,57095.0,1976.0,7245.0,3786.318000,0.0
Afghanistan,1992,8.021577e+05,52776.0,1976.0,5447.0,4028.724000,0.0
Afghanistan,1993,1.092106e+06,3914149.0,1976.0,6465.0,4325.406000,0.0
Afghanistan,1994,8.020524e+05,48965.0,1976.0,6383.0,4656.120000,0.0
...,...,...,...,...,...,...,...
Zimbabwe,2010,3.793025e+04,636386.1,8448.0,84457.0,111946.250685,0.0
Zimbabwe,2011,3.989046e+04,652677.4,8448.0,91158.8,114513.847837,1400.0
Zimbabwe,2012,3.601022e+04,668951.3,8448.0,89050.5,117320.963328,0.0


One thing I need to look into - I think the dataframe differentiates between imports and exports, which I haven't done above